In [ ]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()), 'diffusion_models')
os.chdir(project_root)
%pwd # should be PPGA root dir

In [ ]:
import pickle
import torch
import numpy as np

from autoencoders.policy.transformer import TransformerPolicyAutoencoder
from attrdict import AttrDict
from RL.actor_critic import Actor
from envs.brax_custom.brax_env import make_vec_env_brax
from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image
from dataset.policy_dataset import preprocess_model, postprocess_model

In [ ]:
# params to config
device = torch.device('cuda')
env_name = 'halfcheetah'
seed = 1111
normalize_obs = False
normalize_rewards = True
obs_shape = 18
action_shape = 6
mlp_shape = (128, 128, 6)

env_cfg = AttrDict({
    'env_name': env_name,
    'env_batch_size': None,
    'num_dims': 2,
    'seed': seed,
    'num_envs': 1
})

In [ ]:
archive_df_path = '/home/sumeet/QDPPO/experiments/paper_qdppo_halfcheetah/1111/checkpoints/cp_00002000/archive_00002000.pkl'
with open(archive_df_path, 'rb') as f:
    archive_df = pickle.load(f)

scheduler_path = '/home/sumeet/QDPPO/experiments/paper_qdppo_halfcheetah/1111/checkpoints/cp_00002000/scheduler_00002000.pkl'
with open(scheduler_path, 'rb') as f:
    scheduler = pickle.load(f)

In [ ]:
# make the env
env = make_vec_env_brax(env_cfg)

In [ ]:
def get_best_elite():
    best_elite = scheduler.archive.best_elite
    agent = Actor(obs_shape, action_shape, True, True).deserialize(best_elite.solution).to(device)
    if normalize_obs:
        agent.obs_normalizer = best_elite.metadata['obs_normalizer']
    return agent

In [ ]:
def get_random_elite():
    elite = scheduler.archive.sample_elites(1)
    agent = Actor(obs_shape, action_shape, True, True).deserialize(elite.solution_batch.flatten()).to(device)
    if normalize_obs:
        agent.obs_normalizer = elite.metadata_batch[0]['obs_normalizer']
    return agent

In [ ]:
def enjoy_brax(agent, render=True, deterministic=True):
    if normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(env_cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)

            if deterministic:
                act = agent.actor_mean(obs)
            else:
                act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew
    if render:
        i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
        display(i)
        print(f'{total_reward=}')
        print(f' Rollout length: {len(rollout)}')
        measures /= len(rollout)
        print(f'Measures: {measures.cpu().numpy()}')
    return total_reward.detach().cpu().numpy()

In [ ]:
agent = get_best_elite()
# make sure pre and post-processing are working correctly. This should return
# the exact same agent as the previous line
agent = postprocess_model(agent, preprocess_model(agent, mlp_shape), mlp_shape).to(device)
enjoy_brax(agent, render=True)

In [ ]:
# load the VAE model
autoencoder_cp_path = 'checkpoints/autoencoder.pt'
vae_model = TransformerPolicyAutoencoder(emb_channels=64, z_channels=32)
vae_model.load_state_dict(torch.load(autoencoder_cp_path))
vae_model.to(device)

In [ ]:
# get the policy input tensor
policy_tensor = preprocess_model(agent, (128, 128, 6)).to(device).unsqueeze(dim=0)
policy_tensor.shape

In [ ]:
# get the reconstructed model
out, _ = vae_model(policy_tensor)
out = out.squeeze(dim=0)

model_in = Actor(obs_shape, action_shape, False, True).to(device)
rec_agent = postprocess_model(model_in, out, (128, 128, 6))
rec_agent.to(device)

In [ ]:
enjoy_brax(rec_agent, render=True)